## Esercizio

### Consegna

- Scegliere un verbo transitivo --> **KILL**
- Trovare un corpus con > 1000 frasi in cui comprare un verbo scelto (usare un verbo comune) --> link to resource: https://sentence.yourdictionary.com/kill
- Effettuare parsing e disambiguazione
- Usare i supersensi di wordnet sugli argomenti (subj e obj nel caso di 2 argomenti) del verbo scelto
- Calcolo risultati, frequenza e stampare cluster semantici ottenuti

### Appunti

Wordnet supersense: Per ogni sinset abbiamo un supersenso associato (esempi su slide)
Utilizzare i supersense dei synset per determinare il semantic type - nltk

I supersense di wordenet non sono il massimo, vediamo alcune alternative:
 * CSI - Ai supersense ci sono associati delle categorie che possono essere utilizzati come supersense

link to wordnet resource: https://www.nltk.org/_modules/nltk/corpus/reader/wordnet.html

*Spunti futuri:*

**Come distinguo i synset?**
--> Funzione lesk (https://www.nltk.org/howto/wsd.html#word-sense-disambiguation)

**Come trovo il supersenso tra due termini?**
--> Funzione Lowest Common Hypernyms (https://www.nltk.org/howto/wordnet_lch.html)

#### Wordnet - Esempi di utilizzo di funzioni

#print(f"synsets di subj: {wordnet.synsets(subj)}; synsets di obj: {wordnet.synsets(obj)}\n\n")
#print(f"iperonimo di subj: {wordnet.synsets(subj)[0].hypernyms()}, iperonimo di obj: {wordnet.synsets(obj)[0].hypernyms()}\n\n")
        
print(wordnet.synsets("monkey"))
print(wordnet.synsets("victor")[0].supersense())
print(wordnet.synsets("monkey")[0].definition())
print(wordnet.synsets("monkey")[0].hyponyms())
print("\n\n\n")
print(wordnet.synsets("banana")[0].common_hypernyms(wordnet.synsets("monkey")[0]))
print(wordnet.synsets("dog")[0].root_hypernyms())
print(f"antenato più vicino tra man e baby: {wordnet.synsets('man')[0].lowest_common_hypernyms(wordnet.synsets('baby')[0])}") # UTILE UTILE UTILE

## Code

In [1]:
from nltk.corpus import wordnet
from spacy.matcher import DependencyMatcher
import spacy

* person: Usata per trovare il supersenso dei pronomi
* patter: Usato per trovare il soggetto e l'oggetto del verbo kill

In [3]:
person = ["i", "you", "he", "she", "we", "they", "me", "her", "his", "them", "someone"] 

pattern = [
    {"RIGHT_ID": "attr",
    "RIGHT_ATTRS": {"LEMMA": {"IN": ["kill"]}}
    },
    {"LEFT_ID": "attr",
    "REL_OP": ">",
    "RIGHT_ID": "subj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["nsubj"]}}
    },
    {"LEFT_ID": "attr",
    "REL_OP": ">",
    "RIGHT_ID": "dobj",
    "RIGHT_ATTRS": {"DEP": {"IN": ["dobj"]}}
    }
]

In [7]:
def get_match(name_pattern, pattern, text):
    # Find the pattern in the document
    nlp = spacy.load('en_core_web_sm')
    matched_elements = [] 
    matcher = DependencyMatcher(nlp.vocab)
    matcher.add(name_pattern, [pattern])
    doc = nlp(text.lower())
    matches = matcher(doc)
    matches.sort(key = lambda x : x[1]) #* Probably not needed
    
    for match in matches:
        match_words = sorted(match[1])
        phrase = doc[match_words[0]:match_words[len(match_words)-1]+1]

        subj = phrase[0].text
        dobj = phrase[len(phrase)-1].text


        super_subj, super_dobj = "", ""
        if subj in person:
            super_subj = "Person"
        elif dobj in person:
            super_dobj = "Person"
        
        syn_subj = wordnet.synsets(subj)
        syn_dobj = wordnet.synsets(dobj)
        
        if syn_subj == []:
            print(f"Frase: {text}")
            print(f"Synset non trovato, il soggetto è {subj}")
        elif syn_dobj == []:
            print(f"Frase: {text}")
            print(f"Synset non trovato, l'oggetto' è {dobj}")
        else:
            syn_subj = syn_subj
            syn_dobj = syn_dobj
        '''
        if super_subj != "":
            print(f"S:{subj}: {super_subj}")
        else:
            print(f"{subj}: {syn_subj}")
        
        if super_dobj != "":
            print(f"S:{dobj}: {super_dobj}")
        else:
            print(f"{dobj}: {syn_dobj}")
        '''
        #matched_elements.append(wordnet.synsets(subj)[0].supersense() ,wordnet.synsets(dobj)[0].supersense())
    if matched_elements == []:
        return False
    return matched_elements

In [ ]:
dataset = []
with open ('../sentence_kill.txt', 'r') as f:
    for row in f:
        result = get_match("pattern", pattern, row)
        if result != False:
            dataset.append(result)

### Test 
Utilizzato per codice di prova

Metodo che cerca un match con il pattern riportato sopra. 
Il match che trova è del tipo ("soggetto kill oggetto"), tramite l'accesso alla prima e all'ultima parola riusciamo ad isolare soggetto e oggetto.
Controlliamo se uno dei due (o entrambi) sono presenti nella lista sopra dichiarata, nel caso lo siano, sappiamo già che il loro "supersense" (o iperonimo, dobbiamo capire) è Person.

Due problematiche:
- nel caso in cui abbiamo più di un synset per ogni parola dobbiamo disambiguare e capire quale prendere.
- nel caso ci siano altre parole (Others, nomi di persona,...) che non hanno un synset, dobbiamo capire come gestirli.

In [2]:
def get_match(name_pattern, pattern, text):
    nlp = spacy.load('en_core_web_sm')
    matched_elements = [] 
    matcher = DependencyMatcher(nlp.vocab)
    matcher.add(name_pattern, [pattern])
    doc = nlp(text.lower())
    matches = matcher(doc)
    matches.sort(key = lambda x : x[1])
    for match in matches:
        match_words = sorted(match[1])
        phrase = doc[match_words[0]:match_words[len(match_words)-1]+1]

        subj = phrase[0].text
        dobj = phrase[len(phrase)-1].text


        super_subj, super_dobj = "", ""
        if subj in person:
            super_subj = "Person"
        elif dobj in person:
            super_dobj = "Person"
        
        syn_subj = wordnet.synsets(subj)
        syn_dobj = wordnet.synsets(dobj)
        
        if syn_subj == []:
            print(f"Frase: {text}")
            print(f"Synset non trovato, il soggetto è {subj}")
        elif syn_dobj == []:
            print(f"Frase: {text}")
            print(f"Synset non trovato, l'oggetto' è {dobj}")
        else:
            syn_subj = syn_subj
            syn_dobj = syn_dobj
        '''
        if super_subj != "":
            print(f"S:{subj}: {super_subj}")
        else:
            print(f"{subj}: {syn_subj}")
        
        if super_dobj != "":
            print(f"S:{dobj}: {super_dobj}")
        else:
            print(f"{dobj}: {syn_dobj}")
        '''
        #matched_elements.append(wordnet.synsets(subj)[0].supersense() ,wordnet.synsets(dobj)[0].supersense())
    if matched_elements == []:
        return False
    return matched_elements

# OLD CODE AND NOTES:

Ci sono soggetti e oggetti che non hanno dei synset, ad esempio YOU non ha un wordnet synset, credo che si debbano eliminare dal dataset questi tipi di frase, perchè non si possono trovare gli iperonimi se non hai il synset di partenza. in alternativa, come ha detto il prof a lezione, si può mettere una regola per cui quando incontri uno di questi elementi metti 'person' come iperonimo di default

IL DATASET HA UN BOTTO DI SOGGETTI E OGGETTI CON PRONOMI DI MERDA. UN PÒ BRUTTINO PER I NOSTRI SCOPI

### Esecuzione

In [ ]:
dataset = []
with open ('../sentence_cook.txt', 'r') as f:
    for row in f:
        result = get_match("pattern", pattern, row)
        if result != False:
            dataset.append(result)

### Cose utili sparse

In [ ]:
#print(f"synsets di subj: {wordnet.synsets(subj)}; synsets di obj: {wordnet.synsets(obj)}\n\n")
#print(f"iperonimo di subj: {wordnet.synsets(subj)[0].hypernyms()}, iperonimo di obj: {wordnet.synsets(obj)[0].hypernyms()}\n\n")
        
print(wordnet.synsets("monkey"))
print(wordnet.synsets("victor")[0].supersense())
print(wordnet.synsets("monkey")[0].definition())
print(wordnet.synsets("monkey")[0].hyponyms())
print("\n\n\n")
print(wordnet.synsets("banana")[0].common_hypernyms(wordnet.synsets("monkey")[0]))
print(wordnet.synsets("dog")[0].root_hypernyms())
print(f"antenato più vicino tra man e baby: {wordnet.synsets('man')[0].lowest_common_hypernyms(wordnet.synsets('baby')[0])}") # UTILE UTILE UTILE